In [1]:
import pandas as pd
import numpy as np

In [2]:
from pivot_dataframe import restructure_goal_data

In [3]:
df1 = pd.read_csv("../../data/proc/similarity/daily_similarity.csv")
df2 = pd.read_csv("../../data/proc/goal_selfReport_merged_filtered_wide.csv")

In [4]:
df_new = restructure_goal_data(df1, df2)

Step 1: Pivoting df1 to get both today goals per row...
  Shape after pivot: (7676, 14)

Step 2: Renaming df2 columns...
  Shape: (7788, 13)

Step 3: Merging df1 and df2...
  Shape after merge: (7676, 20)

Step 4: Expanding to 2 rows per day (one per tomorrow goal)...
  Shape after expansion: (15352, 16)

Step 5: Adding max similarity features...

Step 6: Adding tomorrow's completion...
  Final shape: (15352, 25)
  Shape after removing missing tomorrow completions: (14660, 25)


In [5]:
df_new.to_csv("../../data/proc/reg_ready.csv", index=False)
df_new.head(5)

,ParticipantIdentifier,trial_date,tomorrow_goal,today_goal_1,today_goal_2,today_completion_1,today_completion_2,today_effort_1,today_effort_2,today_importance_1,...,which_tomorrow_goal,max_sim_tomorrow_with_today,most_similar_is_goal1,max_sim_same_as_tomorrow,max_sim_goal_completion_today,max_sim_goal_effort_today,max_sim_goal_importance_today,tomorrow_goal_completion,tomorrow_goal_effort,tomorrow_goal_importance
0,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-01,Keep working on psych paper,Keep working on psych paper,Finish chem prelab,71.0,100.0,5.0,3.0,6.0,...,goal_1,1.000000,True,True,71.0,5.0,6.0,100.0,6.0,7.0
1,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-01,Review bio questions,Keep working on psych paper,Finish chem prelab,71.0,100.0,5.0,3.0,6.0,...,goal_2,0.314240,False,False,100.0,3.0,5.0,77.0,4.0,3.0
2,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-02,Finish and hand in psych rough draft,Keep working on psych paper,Review bio questions,100.0,77.0,6.0,4.0,7.0,...,goal_1,0.646013,True,True,100.0,6.0,7.0,100.0,5.0,5.0
3,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-02,Create quick bio lecture notes,Keep working on psych paper,Review bio questions,100.0,77.0,6.0,4.0,7.0,...,goal_2,0.416696,False,False,77.0,4.0,3.0,100.0,6.0,5.0
4,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-03,Practice biology FSG questions,Finish and hand in psych rough draft,Create quick bio lecture notes,100.0,100.0,5.0,6.0,5.0,...,goal_1,0.289335,False,False,100.0,6.0,5.0,90.0,5.0,5.0


In [6]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.regression.mixed_linear_model import MixedLM

# Assuming you have df_new from the restructuring step

def prepare_regression_data(df_new):
    """
    Prepare data for regression analysis
    """
    df_reg = df_new.copy()
    
    # Create binary similarity variable (similar if > median or some threshold)
    # You can adjust the threshold
    threshold = df_reg['max_sim_tomorrow_with_today'].median()
    threshold = 0.80
    df_reg['goal_sim_bin'] = df_reg['max_sim_tomorrow_with_today'] > threshold
    df_reg['llm_same_as_tomorrow'] = df_reg['max_sim_same_as_tomorrow']
    
    # Rename for cleaner formula
    df_reg['next_goal_completion'] = df_reg['tomorrow_goal_completion']
    df_reg['today_goal_completion'] = df_reg['max_sim_goal_completion_today']
    df_reg['goal_sim'] = df_reg['max_sim_tomorrow_with_today']
    df_reg['today_goal_effort'] = df_reg['max_sim_goal_effort_today']
    df_reg['today_goal_importance'] = df_reg['max_sim_goal_importance_today']
    
    # Remove rows with missing values
    df_reg = df_reg.dropna(subset=['next_goal_completion', 'today_goal_completion', 
                                     'goal_sim', 'today_goal_effort'])
    
    return df_reg


def run_model_1(df_reg):
    """
    Model 1: next_goal_completion ~ today_goal_completion * goal_sim_bin
    Tests momentum with binary similarity
    """
    print("="*80)
    print("MODEL 1: Binary Similarity Interaction")
    print("next_goal_completion ~ today_goal_completion * goal_sim_bin")
    print("="*80)
    
    model = MixedLM.from_formula(
        'next_goal_completion ~ today_goal_completion * goal_sim_bin',
        data=df_reg,
        groups=df_reg['ParticipantIdentifier']
    )
    
    result = model.fit(reml=True)
    print(result.summary())
    print("\n")
    
    return result


def run_model_2(df_reg):
    """
    Model 2: next_goal_completion ~ today_goal_completion * goal_sim
    Tests momentum with continuous similarity
    """
    print("="*80)
    print("MODEL 2: Continuous Similarity Interaction")
    print("next_goal_completion ~ today_goal_completion * goal_sim")
    print("="*80)
    
    model = MixedLM.from_formula(
        'next_goal_completion ~ today_goal_completion * goal_sim',
        data=df_reg,
        groups=df_reg['ParticipantIdentifier']
    )
    
    result = model.fit(reml=True)
    print(result.summary())
    print("\n")
    
    return result


def run_model_3(df_reg):
    """
    Model 3: next_goal_completion ~ today_goal_completion * goal_sim * today_goal_effort
    Three-way interaction with effort
    """
    print("="*80)
    print("MODEL 3: Three-way Interaction with Effort")
    print("next_goal_completion ~ today_goal_completion * goal_sim * today_goal_effort")
    print("="*80)
    
    model = MixedLM.from_formula(
        'next_goal_completion ~ today_goal_completion * goal_sim * today_goal_effort',
        data=df_reg,
        groups=df_reg['ParticipantIdentifier']
    )
    
    result = model.fit(reml=True)
    print(result.summary())
    print("\n")
    
    return result


def identify_continuation_goals(df_reg, completion_percentile=90):
    """
    Identify goals that are continuations from previous day
    Based on: same_as_tomorrow==True AND low completion (<95th percentile)
    """
    df_reg = df_reg.copy()
    
    # Calculate 95th percentile of completion for each participant
    participant_90th = df_reg.groupby('ParticipantIdentifier')['today_goal_completion'].quantile(completion_percentile/100)
    
    # Map back to dataframe
    df_reg['completion_90th'] = df_reg['ParticipantIdentifier'].map(participant_90th)
    
    # Create binary indicators
    df_reg['is_same_as_tomorrow'] = df_reg['max_sim_same_as_tomorrow']
    df_reg['low_completion'] = df_reg['today_goal_completion'] < df_reg['completion_90th']
    
    # Continuation = same as tomorrow AND Low completion
    df_reg['is_continuation'] = df_reg['is_same_as_tomorrow'] & df_reg['low_completion']
    
    print(f"Total observations: {len(df_reg)}")
    print(f"Same as tomorrow: {df_reg['is_same_as_tomorrow'].sum()} ({df_reg['is_same_as_tomorrow'].mean()*100:.1f}%)")
    print(f"Low completion (<90th percentile): {df_reg['low_completion'].sum()} ({df_reg['low_completion'].mean()*100:.1f}%)")
    print(f"Continuation goals (both): {df_reg['is_continuation'].sum()} ({df_reg['is_continuation'].mean()*100:.1f}%)")
    print(f"Non-continuation goals: {(~df_reg['is_continuation']).sum()} ({(~df_reg['is_continuation']).mean()*100:.1f}%)")
    
    return df_reg


def test_continuation_effort(df_reg):
    """
    Test if continuation goals have higher effort
    """
    print("="*80)
    print("CONTINUATION vs EFFORT")
    print("today_goal_effort ~ is_continuation")
    print("="*80)
    
    # Create numeric version of is_continuation
    df_reg['cond'] = df_reg['is_continuation'].astype(int)
    
    model = MixedLM.from_formula(
        'today_goal_effort ~ cond',
        data=df_reg,
        groups=df_reg['ParticipantIdentifier']
    )
    
    result = model.fit(reml=True)
    print(result.summary())
    print("\n")
    
    return result


def run_models_without_continuation(df_reg):
    """
    Run all models excluding continuation goals
    """
    df_no_cont = df_reg[~df_reg['is_continuation']].copy()
    
    print("\n" + "="*80)
    print("ANALYSIS EXCLUDING CONTINUATION GOALS")
    print(f"N = {len(df_no_cont)} (removed {len(df_reg) - len(df_no_cont)} continuation goals)")
    print("="*80 + "\n")
    
    result1 = run_model_1(df_no_cont)
    result2 = run_model_2(df_no_cont)
    
    return result1, result2


def full_analysis_pipeline(df_new):
    """
    Complete analysis pipeline
    """
    # Step 1: Prepare data
    print("\n" + "="*80)
    print("STEP 1: PREPARING DATA")
    print("="*80)
    df_reg = prepare_regression_data(df_new)
    print(f"Final sample size: {len(df_reg)} observations")
    print(f"Number of participants: {df_reg['ParticipantIdentifier'].nunique()}")
    print(f"Mean observations per participant: {len(df_reg) / df_reg['ParticipantIdentifier'].nunique():.1f}")
    
    # Step 2: Run models with all data
    print("\n" + "="*80)
    print("STEP 2: MODELS WITH ALL DATA")
    print("="*80 + "\n")
    result1_all = run_model_1(df_reg)
    result2_all = run_model_2(df_reg)
    result3_all = run_model_3(df_reg)
    
    # Step 3: Identify continuation goals
    print("\n" + "="*80)
    print("STEP 3: IDENTIFYING CONTINUATION GOALS")
    print("="*80)
    df_reg = identify_continuation_goals(df_reg)
    
    # Step 4: Test if continuation goals are higher effort
    print("\n")
    result_effort = test_continuation_effort(df_reg)
    
    # Step 5: Run models without continuation goals
    result1_no_cont, result2_no_cont = run_models_without_continuation(df_reg)
    
    # Return everything
    return {
        'data': df_reg,
        'data_no_continuation': df_reg[~df_reg['is_continuation']],
        'model1_all': result1_all,
        'model2_all': result2_all,
        'model3_all': result3_all,
        'effort_test': result_effort,
        'model1_no_cont': result1_no_cont,
        'model2_no_cont': result2_no_cont
    }


# USAGE:
# results = full_analysis_pipeline(df_new)

# Access specific results:
# results['data']  # Full dataset with continuation flags
# results['model1_all']  # Model 1 with all data
# results['model1_no_cont']  # Model 1 without continuation goals

# To see a specific model summary again:
# print(results['model1_all'].summary())


In [7]:
# USAGE:
results = full_analysis_pipeline(df_new)

# Access specific results:
# results['data']  # Full dataset with continuation flags
# results['model1_all']  # Model 1 with all data
# results['model1_no_cont']  # Model 1 without continuation goals

# To see a specific model summary again:
# print(results['model1_all'].summary())


STEP 1: PREPARING DATA
Final sample size: 14660 observations
Number of participants: 112
Mean observations per participant: 130.9

STEP 2: MODELS WITH ALL DATA

MODEL 1: Binary Similarity Interaction
next_goal_completion ~ today_goal_completion * goal_sim_bin
                         Mixed Linear Model Regression Results
Model:                   MixedLM        Dependent Variable:        next_goal_completion
No. Observations:        14660          Method:                    REML                
No. Groups:              112            Scale:                     1192.6146           
Min. group size:         60             Log-Likelihood:            -72879.6014         
Max. group size:         164            Converged:                 Yes                 
Mean group size:         130.9                                                         
---------------------------------------------------------------------------------------
                                            Coef.  Std.Err. 

In [8]:
df_new = df_new.rename(columns={
     'max_sim_goal_completion_today': 'today_goal_completion',
     'max_sim_goal_effort_today': 'today_goal_effort',
     'max_sim_goal_importance_today': 'today_goal_importance',
     'max_sim_same_as_tomorrow': 'same_as_tomorrow',
     'max_sim_tomorrow_with_today': 'goal_sim'
    })

In [ ]:
import pandas as pd

def create_clean_continuation_table(examples_df):
    """
    Create a minimal, readable table for publication
    Shows only essential columns to demonstrate continuation pattern
    Uses most_similar_is_goal1 to determine which goal matched
    """
    
    clean_table = examples_df.copy()
    
    # Create cleaner participant labels
    participant_ids = clean_table['ParticipantIdentifier'].unique()
    participant_map = {pid: f"P{i+1}" for i, pid in enumerate(participant_ids)}
    clean_table['P'] = clean_table['ParticipantIdentifier'].map(participant_map)
    
    # Format date to just MM-DD
    clean_table['Date'] = pd.to_datetime(clean_table['trial_date']).dt.strftime('%m-%d')
    
    # Use most_similar_is_goal1 to determine which goal matched
    clean_table['Today Goal'] = clean_table.apply(
        lambda row: row['today_goal_1'] if row['most_similar_is_goal1'] else row['today_goal_2'],
        axis=1
    )
    
    clean_table['Matched Today Comp%'] = clean_table.apply(
        lambda row: int(row['today_completion_1']) if row['most_similar_is_goal1'] else int(row['today_completion_2']),
        axis=1
    )
    
    # Round similarity
    clean_table['Sim'] = clean_table['goal_sim'].round(2)
    
    # Final selection
    final_table = clean_table[[
        'P',
        'Date',
        'Today Goal',
        'tomorrow_goal',
        'Matched Today Comp%',
        'Sim'
    ]].copy()
    
    final_table.columns = ['P', 'Date', "Today's Goal", "Tomorrow's Goal", 'Matched Today Comp%', 'Sim']
    
    return final_table

def identify_continuation_goals(df_reg, completion_percentile=90):
    """
    Identify goals that are continuations from previous day
    Based on: same_as_tomorrow==True AND high completion (completion_percentile th percentile)
    """
    df_reg = df_reg.copy()
    
    # Calculate 95th percentile of completion for each participant
    participant_95th = df_reg.groupby('ParticipantIdentifier')['today_goal_completion'].quantile(completion_percentile/100)
    
    # Map back to dataframe
    df_reg['completion_95th'] = df_reg['ParticipantIdentifier'].map(participant_95th)
    
    # Create binary indicators
    df_reg['is_same_as_tomorrow'] = df_reg['same_as_tomorrow'] & (df_reg['goal_sim'] > 0.80)
    df_reg['low_completion'] = df_reg['today_goal_completion'] < df_reg['completion_95th']
    
    # Continuation = same as tomorrow AND high completion
    df_reg['is_continuation'] = df_reg['is_same_as_tomorrow'] & df_reg['low_completion']
    
    print(f"Total observations: {len(df_reg)}")
    print(f"Same as tomorrow: {df_reg['is_same_as_tomorrow'].sum()} ({df_reg['is_same_as_tomorrow'].mean()*100:.1f}%)")
    print(f"Low completion (<90th percentile): {df_reg['low_completion'].sum()} ({df_reg['low_completion'].mean()*100:.1f}%)")
    print(f"Continuation goals: {df_reg['is_continuation'].sum()} ({df_reg['is_continuation'].mean()*100:.1f}%)")
    print(f"Non-continuation goals: {(~df_reg['is_continuation']).sum()} ({(~df_reg['is_continuation']).mean()*100:.1f}%)")
    
    return df_reg

In [10]:
df_new = identify_continuation_goals(df_new)
df_new['is_continuation'].value_counts()

Total observations: 14660
Same as tomorrow: 3502 (23.9%)
Low completion (<90th percentile): 10117 (69.0%)
Continuation goals (both): 2793 (19.1%)
Non-continuation goals: 11867 (80.9%)


is_continuation
False    11867
True      2793
Name: count, dtype: int64

In [11]:
continuation_df = df_new[df_new['is_continuation'] == False].copy()
print(len(continuation_df))
continuation_df.head()

11867


,ParticipantIdentifier,trial_date,tomorrow_goal,today_goal_1,today_goal_2,today_completion_1,today_completion_2,today_effort_1,today_effort_2,today_importance_1,...,today_goal_completion,today_goal_effort,today_goal_importance,tomorrow_goal_completion,tomorrow_goal_effort,tomorrow_goal_importance,completion_95th,is_same_as_tomorrow,low_completion,is_continuation
1,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-01,Review bio questions,Keep working on psych paper,Finish chem prelab,71.0,100.0,5.0,3.0,6.0,...,100.0,3.0,5.0,77.0,4.0,3.0,100.0,False,False,False
2,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-02,Finish and hand in psych rough draft,Keep working on psych paper,Review bio questions,100.0,77.0,6.0,4.0,7.0,...,100.0,6.0,7.0,100.0,5.0,5.0,100.0,False,False,False
3,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-02,Create quick bio lecture notes,Keep working on psych paper,Review bio questions,100.0,77.0,6.0,4.0,7.0,...,77.0,4.0,3.0,100.0,6.0,5.0,100.0,False,True,False
4,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-03,Practice biology FSG questions,Finish and hand in psych rough draft,Create quick bio lecture notes,100.0,100.0,5.0,6.0,5.0,...,100.0,6.0,5.0,90.0,5.0,5.0,100.0,False,False,False
5,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-03,Catch up on anthropology readings,Finish and hand in psych rough draft,Create quick bio lecture notes,100.0,100.0,5.0,6.0,5.0,...,100.0,5.0,5.0,54.0,6.0,6.0,100.0,False,False,False


In [12]:
n_examples_per_participant = 5

participant_counts = continuation_df['ParticipantIdentifier'].value_counts()
eligible_participants = participant_counts[participant_counts >= n_examples_per_participant].index.tolist()

np.random.seed(42)  # For reproducibility
selected_participants = np.random.choice(eligible_participants, size=3, replace=False)
    

examples = []
for participant in selected_participants:
    participant_goals = continuation_df[
        continuation_df['ParticipantIdentifier'] == participant
    ].head(n_examples_per_participant)
    examples.append(participant_goals)

examples_df = pd.concat(examples, ignore_index=True)
create_clean_continuation_table(examples_df)

,P,Date,Today's Goal,Tomorrow's Goal,Matched Today Comp%,Sim
0,P1,09-29,Find time for leisure,Catch up on missed lectures,0,0.30
1,P1,09-30,Catch up on missed lectures,Prepare for chemistry midterms,90,0.39
2,P1,09-30,Try to find leisure time,Reduce social media,100,0.24
3,P1,10-04,Avoid procrastinating,Prepare for chemistry midterm,54,0.31
4,P1,10-04,Try to socialize more with people I usually me...,Play more basketball,94,0.35
5,P2,09-29,I want to start studying for my midterm,I want to finish bio homework.,20,0.48
6,P2,09-29,I want to start studying for my midterm,I want to catch up in one of my classes,20,0.55
7,P2,09-30,I want to finish bio homework.,I want to finish my chemistry paragraph,100,0.60
8,P2,09-30,I want to catch up in one of my classes,I want to sleep more than 3 hours,29,0.30
9,P2,10-01,I want to finish my chemistry paragraph,i want to finish my psychology paragraph,75,0.70


In [13]:
def run_model_1(df_reg):
    """
    Model 1: tomorrow_goal_completion ~ today_goal_completion * same_as_tomorrow
    Tests momentum with binary similarity
    """
    print("="*80)
    print("MODEL 1: Binary Similarity Interaction")
    print("tomorrow_goal_completion ~ today_goal_completion * same_as_tomorrow")
    print("="*80)
    
    model = MixedLM.from_formula(
        'tomorrow_goal_completion ~ today_goal_completion * same_as_tomorrow',
        data=df_reg,
        groups=df_reg['ParticipantIdentifier']
    )
    
    result = model.fit(reml=True)
    print(result.summary())
    print("\n")
    
    return result


def run_model_2(df_reg):
    """
    Model 2: tomorrow_goal_completion ~ today_goal_completion * goal_sim
    Tests momentum with continuous similarity
    """
    print("="*80)
    print("MODEL 2: Continuous Similarity Interaction")
    print("tomorrow_goal_completion ~ today_goal_completion * goal_sim")
    print("="*80)
    
    model = MixedLM.from_formula(
        'tomorrow_goal_completion ~ today_goal_completion * goal_sim',
        data=df_reg,
        groups=df_reg['ParticipantIdentifier']
    )
    
    result = model.fit(reml=True)
    print(result.summary())
    print("\n")
    
    return result


In [14]:
high_sim = continuation_df.loc[continuation_df['goal_sim'] > 0.80]
low_sim = continuation_df.loc[continuation_df['goal_sim'] <= 0.80]

run_model_1(low_sim)
run_model_2(low_sim)

MODEL 1: Binary Similarity Interaction
tomorrow_goal_completion ~ today_goal_completion * same_as_tomorrow
                          Mixed Linear Model Regression Results
Model:                    MixedLM       Dependent Variable:       tomorrow_goal_completion
No. Observations:         11046         Method:                   REML                    
No. Groups:               112           Scale:                    1276.4894               
Min. group size:          1             Log-Likelihood:           -55309.8103             
Max. group size:          155           Converged:                Yes                     
Mean group size:          98.6                                                            
------------------------------------------------------------------------------------------
                                                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------------
Intercept 

___
## JUST LOOK AT goal_sim ~ today_goal_completion
___

In [15]:
df_new.head()

,ParticipantIdentifier,trial_date,tomorrow_goal,today_goal_1,today_goal_2,today_completion_1,today_completion_2,today_effort_1,today_effort_2,today_importance_1,...,today_goal_completion,today_goal_effort,today_goal_importance,tomorrow_goal_completion,tomorrow_goal_effort,tomorrow_goal_importance,completion_95th,is_same_as_tomorrow,low_completion,is_continuation
0,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-01,Keep working on psych paper,Keep working on psych paper,Finish chem prelab,71.0,100.0,5.0,3.0,6.0,...,71.0,5.0,6.0,100.0,6.0,7.0,100.0,True,True,True
1,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-01,Review bio questions,Keep working on psych paper,Finish chem prelab,71.0,100.0,5.0,3.0,6.0,...,100.0,3.0,5.0,77.0,4.0,3.0,100.0,False,False,False
2,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-02,Finish and hand in psych rough draft,Keep working on psych paper,Review bio questions,100.0,77.0,6.0,4.0,7.0,...,100.0,6.0,7.0,100.0,5.0,5.0,100.0,False,False,False
3,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-02,Create quick bio lecture notes,Keep working on psych paper,Review bio questions,100.0,77.0,6.0,4.0,7.0,...,77.0,4.0,3.0,100.0,6.0,5.0,100.0,False,True,False
4,0151d9f1-1644-4437-805e-02f5e244a690,2023-02-03,Practice biology FSG questions,Finish and hand in psych rough draft,Create quick bio lecture notes,100.0,100.0,5.0,6.0,5.0,...,100.0,6.0,5.0,90.0,5.0,5.0,100.0,False,False,False


In [16]:
model = MixedLM.from_formula(
        'goal_sim ~ today_goal_completion ',
        data=df_new,
        groups=df_new['ParticipantIdentifier']
    )
    
result = model.fit(reml=True)
print(result.summary())
print("\n")

              Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    goal_sim  
No. Observations:    14660      Method:                REML      
No. Groups:          112        Scale:                 0.0654    
Min. group size:     60         Log-Likelihood:        -1010.9930
Max. group size:     164        Converged:             Yes       
Mean group size:     130.9                                       
-----------------------------------------------------------------
                      Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-----------------------------------------------------------------
Intercept              0.616    0.012  51.055 0.000  0.593  0.640
today_goal_completion -0.001    0.000 -19.970 0.000 -0.001 -0.001
Group Var              0.014    0.008                            



